### Домашняя работа № 2

**Цель работы:** научиться использовать Python для загрузки данных
из интернет с помощью парсинга Web-страниц и через REST API.

В этой работе мы будем загружать данные о вакансиях
разработчиков Python с сайта hh.ru.

**Задание**
Разработайте программу на Python, которая скачивает
информацию о вакансиях разработчиков Python среднего уровня
(middle python developer).

Программа должна получать следующую информацию о каждой
вакансии:
* Название компании.
* Название позиции (должности).
* Описание вакансии.
* Список ключевых навыков.

Программа должна получать данные о вакансиях двумя
способами:
1. Парсинг Web-страниц сайта hh.ru.
2. Вызов API hh.ru (https://dev.hh.ru/).

Программа должна скачать информацию о 100 вакансиях.

Скачанные данные о вакансиях запишите в базу данных SQLite, таблица
vacancies, столбцы:
* company_name (название компании).
* position (название позиции).
* job_description (описание вакансии).
* key_skills (список ключевых навыков).

**Задание повышенной сложности:** программа должна загружать данные о
вакансиях асинхронно.

Для сдачи задания пришлите исходные коды программ на Python или
ссылку на git репозиторий с исходным кодом.

### Решение

In [1]:
import pandas as pd
# from typing import List
# import requests
from time import sleep
# import sqlite3
# from pathlib import Path
from utils import (areas_parser, get_query, execute, get_data_by_api,
                   get_table, get_view, data_parser, persist_df, normalizer)

In [2]:
db_name = 'hw2.db'

url_areas = 'https://api.hh.ru/areas'
url_vac = "https://api.hh.ru/vacancies"

country = 'Россия'
regions = ['Москва', 'Санкт-Петербург', 'Краснодарский край']


areas_dct = areas_parser(url_areas, country, regions)  # получаем id заданных регионов
print(f'id регионов: {areas_dct}')
areas_ids = list(areas_dct)

url_params = {
    "text": "middle python",
    "search_field": "name",
    # "found": 200,
    # "pages": 4,
    "per_page": 50,
    "page": 0,
    "area": areas_ids,
    "archived": False
}

id регионов: {1: 'Москва', 2: 'Санкт-Петербург', 1438: 'Краснодарский край'}


In [4]:
def get_vacancies(url, params, vacancies=None):
    """Получает список вакансий по заданному URL с параметрами"""
    if vacancies is None:
        vacancies = []

    if params.get('page', 0) < params.get('pages', 1):
        data = get_data_by_api(url, params)
        _vacancies = data.get('items', [])
        found, page, pages = data.get('found', 0), data.get('page', 0), data.get('pages', 1)
        params.update(found=found, page=page + 1, pages=pages)
        print(f'\tfound: {found}, pages: {pages}, page: {page} in get_vac')
        vacancies += _vacancies

        print('\tpage after in get_vac', params.get('page'), page)

        # Количество вакансий
        print('Количество спарсенных со стр', page,
              'вакансий по заданным параметрам:', len(_vacancies),
              'всего спарсено вакансий:', len(vacancies), end='\n\n')

        if len(vacancies) < (found if found < 100 else 100) and page < pages:
            sleep(3)
            get_vacancies(url, params, vacancies)
            page += 1
            print('\tpage after recursion in get_vac', params.get('page'), page)

    print('\tpage final in get_vac', params.get('page'))
    return vacancies


def attributes_processing(vacancies_df: pd.DataFrame):
    """Обработка DF вакансий"""
    # Парсим атрибуты, достаём названия у тех, которые имеют форму словарей/списков
    # и добавляем в DF вакансий
    for attrib in vacancies_df:
        if attrib in ('employer', 'salary'):
            continue
        print('Parsing attribute', attrib)
        data = vacancies_df[attrib].apply(data_parser)
        if isinstance(data, pd.DataFrame):
            if 'name' in data:
                vacancies_df[attrib] = data['name']
        else:
            vacancies_df[attrib] = data
    print()


def vacancies_processing(df: pd.DataFrame = pd.DataFrame()):
    """Обработка списка вакансий и преобразование в таблицы"""
    print('page before in proc', url_params.get('page'))
    fill_vac = [{}] * len(df)
    vacancies = get_vacancies(url_vac, url_params, fill_vac)
    found, page, pages = (url_params.get('found', 0),
                          url_params.get('page', 0),
                          url_params.get('pages', 0))
    print('page after in proc', page)
    print('num vac =', len(vacancies))
    print('len df is:', len(df))

    # Парсим список вакансий в DF и оставляем только нужные атрибуты
    vacancy_attribs = ['id', 'name', 'url', 'alternate_url', 'employer', 'area',
                       'employment', 'salary', 'experience', 'professional_roles',
                       'published_at', 'created_at', 'archived']
    global vacancies_df
    vacancies_df = pd.DataFrame(vacancies)[vacancy_attribs]
    # очистка DF от добавленных пустых строк
    vacancies_df = vacancies_df.dropna(how='all').reset_index(drop=True)
    vacancies.clear()  # очистка массива со спарсенными ранее вакансиями
    vacancies_df.rename(columns=dict(name='position'), inplace=True)
    # Убираем архивные вакансии
    vacancies_df = vacancies_df[~vacancies_df.archived.fillna(False)]

    # Обрабатываем атрибуты
    attributes_processing(vacancies_df)

    # Парсим работодателей в DF для создания отдельной таблицы
    employers_df = vacancies_df.employer.apply(data_parser)
    # Убираем лишний атрибут logo_urls
    employers_df.drop(columns='logo_urls', inplace=True)

    vacancies_df = pd.concat([vacancies_df,
                              employers_df.add_prefix('company_')], axis=1)
    # Убираем дубли
    employers_df = employers_df[~employers_df.duplicated()]
    # # #
    # TODO: доделать этот код
    # nrows = execute(get_query('delete_from_employers.sql'),
    #                 db_name, dict(id=employers_df.id, name=employers_df.name))
    # print('Удалено', nrows, 'строк(и) из таблицы employers')
    # # #

    # Запись DF в таблицу БД employers
    persist_df(employers_df, 'employers', db_name, if_exists='append', attempts=3)

    # Парсим зарплаты
    salary_df = vacancies_df.salary.apply(data_parser)
    vacancies_df = pd.concat([vacancies_df,
                              salary_df.add_prefix('salary_')], axis=1)

    # Парсим дополнительные атрибуты по каждой вакансии по API и записываем в DF вакансий
    vacancies_df['details'] = vacancies_df.url.apply(get_data_by_api)

    details_df = vacancies_df.details.apply(data_parser)
    details_df = details_df[['description',
                             'key_skills']].rename(columns=dict(description='job_description'))

    vacancies_df = pd.concat([vacancies_df, details_df], axis=1)
    # vacancies_df.drop(columns='details', inplace=True)

    # Обработка атрибута key_skills
    def func(x):
        # print(x)
        if isinstance(x, list):
            return ', '.join([v['name'] for v in x]) if x else None
        else:
            return x

    vacancies_df['key_skills'] = vacancies_df.key_skills.apply(func)
    # Оставляем только те вакансии, в которых указаны ключевые навыки,
    # которые разместили проверенные работодатели,имеющие аккредитацию IT компании
    vacancies_df = vacancies_df[vacancies_df.company_trusted &
                                # vacancies_df.company_accredited_it_employer &
                                vacancies_df.key_skills.notna()]
    # убираем лишние атрибуты
    vacancies_attribs = [
        'id',
        'position',
        'job_description',
        'url',
        'alternate_url',
        'area',
        'employment',
        'experience',
        'professional_roles',
        'key_skills',
        'salary_from',
        'salary_to',
        'salary_currency',
        'salary_gross',
        'company_id',
        'company_name',
        'published_at',
        'created_at',
        'archived'
    ]
    vacancies_df = pd.concat([df, vacancies_df[vacancies_attribs]], ignore_index=True)
    print('len vacancies_df after filters is:', len(vacancies_df))
    if len(vacancies_df) < (found if found < 100 else 100) and page < pages:
        # Если количество вакансий после фильтров оказалось меньше 100,
        # и при этом найдено более 100 вакансий, то запускаем процесс заново
        vacancies_processing(vacancies_df.copy(deep=True))
        print('len vacancies_df after recursion is:', len(vacancies_df))
    print('len vacancies_df final is:', len(vacancies_df))

    # Запись DF в таблицу БД vacancies
    persist_df(vacancies_df, 'vacancies', db_name, if_exists='append', attempts=3)

    # Сздаём отдельный DF с ключевыми скиллами
    key_skills_df = vacancies_df[['id', 'key_skills']].rename(columns=dict(id='vacancy_id',
                                                                       key_skills='name'))
    # Разбиваем ключевые скиллы, записанные через запятую на списки
    key_skills_df['name'] = key_skills_df.name.apply(lambda x: x.split(',')
                                                     if isinstance(x, str)
                                                     else x)
    # Растягиваем списки на атомарные значения после разбивки
    key_skills_df = key_skills_df.explode('name', ignore_index=True)
    # Нормализуем ключевые слова
    key_skills_df['normalized_name'] = key_skills_df.name.apply(normalizer)

    # Запись DF в таблицу БД key_skills
    persist_df(key_skills_df, 'key_skills', db_name, if_exists='append', attempts=3)
    # Очистка БД от мусора
    execute('vacuum', db_name)


# Создаём таблицу employers в БД hw2.db
execute(get_query('create_employers.sql'), db_name)
# Создаём таблицу vacancies в БД hw2.db
execute(get_query('create_vacancies.sql'), db_name)
# Создаём таблицу key_skills в БД hw2.db
execute(get_query('create_key_skills.sql'), db_name)

vacancies_processing()

page before in proc 0
URL: https://api.hh.ru/vacancies?text=middle+python&search_field=name&per_page=50&page=0&area=1&area=2&area=1438&archived=False
GET request sucessful
	found: 124, pages: 3, page: 0 in get_vac
	page after in get_vac 1 0
Количество спарсенных со стр 0 вакансий по заданным параметрам: 50 всего спарсено вакансий: 50

URL: https://api.hh.ru/vacancies?text=middle+python&search_field=name&per_page=50&page=1&area=1&area=2&area=1438&archived=False&found=124&pages=3
GET request sucessful
	found: 124, pages: 3, page: 1 in get_vac
	page after in get_vac 2 1
Количество спарсенных со стр 1 вакансий по заданным параметрам: 50 всего спарсено вакансий: 100

	page final in get_vac 2
	page after recursion in get_vac 2 1
	page final in get_vac 2
page after in proc 2
num vac = 100
len df is: 0
Parsing attribute id
Parsing attribute position
Parsing attribute url
Parsing attribute alternate_url
Parsing attribute area
Parsing attribute employment
Parsing attribute experience
Parsing att

### Проверка результата

In [5]:
# Проверяем получившийся результат по компаниям
get_table('employers', db_name)

,id,name,url,alternate_url,vacancies_url,accredited_it_employer,trusted
0,5843588.0,Webtronics,https://api.hh.ru/employers/5843588,https://hh.ru/employer/5843588,https://api.hh.ru/vacancies?employer_id=5843588,0.0,1
1,NaN,Перспективный стартап,None,None,None,NaN,1
2,10044600.0,MoneyCat,https://api.hh.ru/employers/10044600,https://hh.ru/employer/10044600,https://api.hh.ru/vacancies?employer_id=10044600,0.0,1
3,3202769.0,ДЖАСТ ВОРК,https://api.hh.ru/employers/3202769,https://hh.ru/employer/3202769,https://api.hh.ru/vacancies?employer_id=3202769,0.0,1
4,5430966.0,Divo.ai,https://api.hh.ru/employers/5430966,https://hh.ru/employer/5430966,https://api.hh.ru/vacancies?employer_id=5430966,0.0,1
...,...,...,...,...,...,...,...
98,1898886.0,Тазмар,https://api.hh.ru/employers/1898886,https://hh.ru/employer/1898886,https://api.hh.ru/vacancies?employer_id=1898886,0.0,1
99,656166.0,NGENIX,https://api.hh.ru/employers/656166,https://hh.ru/employer/656166,https://api.hh.ru/vacancies?employer_id=656166,1.0,1
100,976931.0,The Skolkovo Institute of Science and Technology,https://api.hh.ru/employers/976931,https://hh.ru/employer/976931,https://api.hh.ru/vacancies?employer_id=976931,0.0,1
101,9201947.0,Betby,https://api.hh.ru/employers/9201947,https://hh.ru/employer/9201947,https://api.hh.ru/vacancies?employer_id=9201947,0.0,1


In [6]:
# Проверяем получившийся результат по вакансиям
get_table('vacancies', db_name)

,id,position,job_description,url,alternate_url,area,employment,experience,professional_roles,key_skills,salary_from,salary_to,salary_currency,salary_gross,company_id,company_name,published_at,created_at,archived
0,55088580,"Разработчик Python (Middle, Senior)",<p>NGENIX — лидер на рынке облачных решений и ...,https://api.hh.ru/vacancies/55088580?host=hh.ru,https://hh.ru/vacancy/55088580,Москва,Полная занятость,От 3 до 6 лет,"Программист, разработчик","Python, PostgreSQL, SQL, MySQL, Scrum",NaN,NaN,None,NaN,656166.0,NGENIX,2023-07-23T09:57:45+0300,2023-07-23T09:57:45+0300,0
1,67115536,Middle Python разработчик (Django),<p><em>МедРокет</em> — платформа IT-решений дл...,https://api.hh.ru/vacancies/67115536?host=hh.ru,https://hh.ru/vacancy/67115536,Краснодар,Полная занятость,От 1 года до 3 лет,"Программист, разработчик","Python, Django Framework",160000.0,240000.0,RUR,1.0,1545815.0,МедРокет,2023-07-18T08:43:43+0300,2023-07-18T08:43:43+0300,0
2,71226010,Middle Python backend developer (ML Space AI C...,<p>Приглашаем <strong>Middle Backend Developer...,https://api.hh.ru/vacancies/71226010?host=hh.ru,https://hh.ru/vacancy/71226010,Москва,Полная занятость,От 1 года до 3 лет,"Программист, разработчик","Backend, Middle, Kubernetes, Python, Linux, Go...",NaN,NaN,None,NaN,3853446.0,Cloud.ru,2023-06-30T22:38:52+0300,2023-06-30T22:38:52+0300,0
3,71226025,Middle/Senior Python Developer (Platform Virtu...,<p><strong>Вам предстоит:</strong></p> <ul> <l...,https://api.hh.ru/vacancies/71226025?host=hh.ru,https://hh.ru/vacancy/71226025,Москва,Полная занятость,От 3 до 6 лет,"Программист, разработчик","Python, Linux, SQL, ООП, Английский язык, Kube...",NaN,NaN,None,NaN,3853446.0,Cloud.ru,2023-06-30T22:38:29+0300,2023-06-30T22:38:29+0300,0
4,72687504,Middle Python developer,<p>Мы занимаемся разработкой и поддержкой собс...,https://api.hh.ru/vacancies/72687504?host=hh.ru,https://hh.ru/vacancy/72687504,Москва,Полная занятость,От 3 до 6 лет,"Программист, разработчик","Python, Linux, MySQL, Nginx, SQL, PostgreSQL, ...",150000.0,200000.0,RUR,0.0,4920624.0,Селфсек,2023-07-21T19:16:41+0300,2023-07-21T19:16:41+0300,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,83781565,Python developer (Middle),<p>Ищу<strong> Python developer </strong>для к...,https://api.hh.ru/vacancies/83781565?host=hh.ru,https://hh.ru/vacancy/83781565,Москва,Полная занятость,От 3 до 6 лет,"Программист, разработчик","Python, Linux, FastAPI, SQLAlchemy",NaN,NaN,None,NaN,1111058.0,"Hi, Rockits!",2023-07-20T19:35:13+0300,2023-07-20T19:35:13+0300,0
102,83808487,Middle/Senior Python developer,<p><strong><em>Информация о компании: </em></s...,https://api.hh.ru/vacancies/83808487?host=hh.ru,https://hh.ru/vacancy/83808487,Москва,Полная занятость,От 3 до 6 лет,"Программист, разработчик","Git, PostgreSQL, Управление проектами, Docker,...",NaN,NaN,None,NaN,5569859.0,ARK,2023-07-21T12:45:50+0300,2023-07-21T12:45:50+0300,0
103,83810414,Middle/Senior AQA Mobile (Python),<p>Наша компания разрабатывает платформу МойОф...,https://api.hh.ru/vacancies/83810414?host=hh.ru,https://hh.ru/vacancy/83810414,Санкт-Петербург,Полная занятость,От 3 до 6 лет,Тестировщик,"Python, UI, ООП, CI",NaN,NaN,None,NaN,213397.0,МойОфис,2023-07-21T13:10:02+0300,2023-07-21T13:10:02+0300,0
104,83827949,Python developer (middle/middle+),<p><strong>Привет! Мы ищем человека в команду ...,https://api.hh.ru/vacancies/83827949?host=hh.ru,https://hh.ru/vacancy/83827949,Санкт-Петербург,Полная занятость,От 3 до 6 лет,"Программист, разработчик","Python, PostgreSQL, Docker, Базы данных, Unit ...",NaN,NaN,None,NaN,1504670.0,HEADS and HANDS,2023-07-21T18:05:30+0300,2023-07-21T18:05:30+0300,0


In [16]:
# Проверяем получившийся результат по ключевым навыкам
key_skills_df = get_table('key_skills', db_name)
key_skills_df

,id,vacancy_id,name,normalized_name
0,1,82841724,Git,git
1,2,82841724,Redis,redis
2,3,82841724,PostgreSQL,sql
3,4,82841724,Python,python
4,5,82841724,REST,rest api
...,...,...,...,...
1581,1582,82510263,Kubernetes,kubernetes
1582,1583,82510263,Docker,docker
1583,1584,82973979,Python,python
1584,1585,82973979,Robot Framework,robot


In [15]:
# Проверяем сколько всего вакансий, соответствующих требованиям спарсилось в БД
stmt_total_count = get_query('count_total_vacancies.sql')

get_view(stmt_total_count, db_name)

,cnt_vac
0,106


### Топ-10 самых востребованных навыков на позицию middle python developer

In [17]:
# Подсчёт методами Pandas
key_skills_df.normalized_name.value_counts().to_frame('counts').head(10)

,counts
python,218
sql,206
rest api,110
django,102
git,82
docker,80
linux,66
flask,38
redis,36
rabbitmq,36


In [19]:
# Топ-10 самых востребованных ключевых навыков на позицию middle_python (запросом)
stmt_top10 = get_query('calculate_top10_key_skills.sql')
get_view(stmt_top10, db_name)

,normalized_name,counts
0,python,218
1,sql,206
2,rest api,110
3,django,102
4,git,82
5,docker,80
6,linux,66
7,flask,38
8,redis,36
9,rabbitmq,36


#### Атрибуты для парсинга

company_name (название компании). employer {name, url, alternate_url} \
position (название позиции). name \
job_description (описание вакансии). snippet | description \
key_skills (список ключевых навыков). key_skills [name]
- area {name}
- url
- alternate_url
- published_at
- created_at
- archived == False
- salary {from, to, currency, gross}
- responsibility
- experience {name}
- employment {name}
- accredited_it_employer == True
- professional_roles {name}